<a href="https://colab.research.google.com/github/Hlamulo1/Federated-research/blob/main/Federated_learning_fix_python_execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install TensorFlow Federated
!pip install tensorflow-federated

import tensorflow as tf
import tensorflow_federated as tff
import numpy as np

# The rest of your federated learning code...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np

# Load and preprocess the dataset
# You need to replace this with your federated dataset loading code
# Assume you have federated_train_data, federated_test_data, and train_data_type

# Define a TFF model function
def create_federated_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(54,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return tff.learning.from_keras_model(model, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])

# Federated Learning setup
tff.backends.native.set_local_python_execution_context()

train_data_type = tff.to_type(train_data[0].element_spec)

@tff.tf_computation(train_data_type)
def initialize_fn(train_data):
    model = create_federated_model()
    return tff.learning.from_keras_model(model, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])

@tff.federated_computation
def next_fn(server_weights, federated_dataset):
    model = create_federated_model()
    server_weights.assign_weights_to(model)
    return tff.learning.build_federated_averaging_process(model, server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01))

iterative_process = tff.templates.IterativeProcess(initialize_fn=initialize_fn, next_fn=next_fn)

state = iterative_process.initialize(federated_train_data[0])

NUM_ROUNDS = 10
for round_num in range(NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f'Round {round_num}: {metrics}')

# Federated evaluation
def federated_evaluate(model, test_data):
    test_metrics = tff.learning.build_federated_evaluation(model)
    return test_metrics(test_data)

federated_model = create_federated_model()
federated_eval_metrics = federated_evaluate(federated_model, federated_test_data)
print(f'Federated Evaluation: {federated_eval_metrics}')


AttributeError: ignored